# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

In [6]:
%load_ext dotenv
%dotenv 
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
from logger import get_logger
_logs = get_logger(__name__)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [7]:
# Importing the relevant libraries 

import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from glob import glob

c:\Users\david\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\david\AppData\Local\Temp\ipykernel_18976\1724091155.py:3: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can b

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [18]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [19]:
adult_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  int32 
dtypes: int32(1), int64(6), object(8)
memory usage: 3.6+ MB


In [25]:
adult_dt = adult_dt.rename(
    columns= {
        'age' : 'age', 
        'workclass' : 'workclass', 
        'fnlwgt' : 'fnlwgt',
        'education': 'education',
        'education-num' : 'education_level', 
        'marital-status' : 'marital_status', 
        'occupation' : 'occupation', 
        'relationship' : 'relationship', 
        'race' : 'race', 
        'sex' : 'sex', 
        'capital-gain' : 'capital_gain', 
        'capital-loss': 'capital_loss', 
        'hours-per-week' : 'hours_per_week', 
        'native-country' : 'native_country', 
        'income' : 'income'
    }
)

In [26]:
adult_dt.describe()

,age,fnlwgt,education_level,capital_gain,capital_loss,hours_per_week,income
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456,0.240810
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429,0.427581
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [27]:
adult_dt.head()

,age,workclass,fnlwgt,education,education_level,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [28]:
X = adult_dt.drop(columns= 'income')
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42) 

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The random state helps us introduce randomness into the code while maintaining reproducibility to ensure that our code is indeed working the way we need it to. 

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [77]:
num_cols = ['age','fnlwgt','education_level','capital_gain',
            'capital_loss', 
]

cat_cols = ['workclass', 'education', 'marital_status', 
            'occupation', 'relationship', 'race', 'sex', 'native_country']

pipe_num_simple = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')), 
    ('standardizer', RobustScaler())
])

pipe_num_yj = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(drop='if_binary' , handle_unknown='infrequent_if_exist')),
])

transformer = ColumnTransformer(
    transformers=[
        ('num_transforms', pipe_num_simple, num_cols), 
        ('cat_transforms', pipe_num_yj, cat_cols)
    ])

transformer

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('standardizer',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education_level',
                                  'capital_gain', 'capital_loss']),
                                ('cat_transforms',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['workclass', 'education', 'marital_status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native_country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [58]:
pipe_simple = Pipeline([
    ('preprocess', transformer), 
    ('classifier', RandomForestClassifier())
])

pipe_simple

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('standardizer',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education_level',
                                                   'capital_gain',
                                                   'capital_loss']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital_status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native_country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [59]:
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

res_simple_dict_1 = cross_validate(pipe_simple, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)

res_simple = pd.DataFrame(res_simple_dict_1).assign(experiment = 1)
res_simple



c:\Users\david\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\david\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,experiment
0,13.586199,0.321185,-0.397718,-0.082023,0.893140,0.999999,0.842290,0.999890,0.766831,0.999851,1
1,14.028331,0.248136,-0.396229,-0.082291,0.894949,0.999998,0.840974,0.999890,0.760738,0.999851,1
2,12.916782,0.231899,-0.442136,-0.082391,0.887564,1.000000,0.843352,0.999945,0.762152,0.999887,1
3,13.666020,0.254627,-0.400488,-0.082985,0.897947,1.000000,0.850812,0.999890,0.774152,0.999851,1
4,13.508126,0.255632,-0.416280,-0.082394,0.893770,1.000000,0.850592,1.000000,0.772469,1.000000,1


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [79]:
cv_results_sorted = res_simple.sort_values('test_neg_log_loss', ascending = False)
cv_results_sorted

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,experiment
1,14.028331,0.248136,-0.396229,-0.082291,0.894949,0.999998,0.840974,0.999890,0.760738,0.999851,1
0,13.586199,0.321185,-0.397718,-0.082023,0.893140,0.999999,0.842290,0.999890,0.766831,0.999851,1
3,13.666020,0.254627,-0.400488,-0.082985,0.897947,1.000000,0.850812,0.999890,0.774152,0.999851,1
4,13.508126,0.255632,-0.416280,-0.082394,0.893770,1.000000,0.850592,1.000000,0.772469,1.000000,1
2,12.916782,0.231899,-0.442136,-0.082391,0.887564,1.000000,0.843352,0.999945,0.762152,0.999887,1


Calculate the mean of each metric. 

In [78]:
cv_results_sorted.mean()

fit_time                   13.541092
score_time                  0.262296
test_neg_log_loss          -0.410570
train_neg_log_loss         -0.082417
test_roc_auc                0.893474
train_roc_auc               0.999999
test_accuracy               0.845604
train_accuracy              0.999923
test_balanced_accuracy      0.767269
train_balanced_accuracy     0.999888
experiment                  1.000000
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [81]:
res_simple_dict_2 = cross_validate(pipe_simple, X_test, Y_test, cv=5, scoring=scoring, return_train_score=True)

res_simple_2 = pd.DataFrame(res_simple_dict_2).assign(experiment = 2)
res_simple_2


c:\Users\david\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\david\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,experiment
0,4.359771,0.141057,-0.410488,-0.086512,0.896932,0.999994,0.851075,0.999744,0.769881,0.999646,2
1,4.298801,0.127828,-0.362163,-0.085613,0.897187,0.999999,0.850563,0.999872,0.762100,0.999916,2
2,4.034185,0.120999,-0.436855,-0.084696,0.880639,1.000000,0.834698,0.999872,0.746493,0.999730,2
3,4.068786,0.115458,-0.450295,-0.084047,0.882360,1.000000,0.845445,1.000000,0.754279,1.000000,2
4,4.026201,0.137872,-0.406993,-0.083854,0.878266,1.000000,0.838198,0.999872,0.760335,0.999916,2


In [75]:
cv_results_sorted_2 = res_simple_2.sort_values('test_neg_log_loss', ascending = False)
cv_results_sorted_2

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,experiment
1,4.057617,0.114794,-0.349450,-0.085536,0.896144,1.000000,0.843910,0.999872,0.757741,0.999916,2
0,4.089597,0.110843,-0.410894,-0.085655,0.896675,0.999998,0.852610,0.999616,0.772377,0.999376,2
4,4.155098,0.104357,-0.430030,-0.084123,0.873622,1.000000,0.830517,0.999872,0.746342,0.999916,2
3,4.097792,0.107355,-0.437469,-0.083112,0.880998,1.000000,0.845957,1.000000,0.755359,1.000000,2
2,3.929665,0.108452,-0.481081,-0.084436,0.881026,1.000000,0.831627,1.000000,0.740758,1.000000,2


In [76]:
cv_results_sorted_2.mean()

fit_time                   4.065954
score_time                 0.109160
test_neg_log_loss         -0.421785
train_neg_log_loss        -0.084572
test_roc_auc               0.885693
train_roc_auc              1.000000
test_accuracy              0.840924
train_accuracy             0.999872
test_balanced_accuracy     0.754515
train_balanced_accuracy    0.999842
experiment                 2.000000
dtype: float64

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

The benefit is that it enables Scikit Learn's binary classification. Simplifies metric calculations. ROC AUC requires the label of interest to be encoded as 1. 

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.